In [1]:
import pandas as pd
from pathlib import Path
import json
import yaml

In [5]:
folder_to_havest = Path("../data/douhet")

json_paths = folder_to_havest.glob("**/*.json")
json_paths_with_labels = zip(map(lambda x: x.stem, json_paths), json_paths)


db = pd.DataFrame()

for label, json_path in json_paths_with_labels:
    with open(json_path, "r") as fin:
        
        data = json.loads(fin.read())
        root_key = list(data.keys())[0]
        
        sna_micro_stats_a = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_stats_b = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_sociogram = data[root_key]["sociogram"]["micro_stats"]
        
        sna_micro_stats_a_df = pd.DataFrame(sna_micro_stats_a).T.add_suffix("_a", axis=1)
        sna_macro_stats_b_df = pd.DataFrame(sna_micro_stats_b).T.add_suffix("_b", axis=1)
        sociogram_micro_stats_df = pd.DataFrame(sna_micro_sociogram).T
        
        course_df =  pd.concat([sna_micro_stats_a_df, sna_macro_stats_b_df, sociogram_micro_stats_df], axis=1)
        course_df = course_df.reset_index(names="id")
        course_df.insert(0, "corso", label)
        db = pd.concat([db, course_df])

db.to_csv("data_abgrid_per_analisi.xlsx", index=False)